In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from finance.trading.factor.benchmark import *
from finance.trading.factor.proj_grad import *

/home/georges/github/finance-bot/python/src/finance/trading/factor/proj_grad.py:127: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 2, 'C', False, aligned=True))
  returns
/home/georges/github/finance-bot/python/src/finance/trading/factor/proj_grad.py:130: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 2, 'C', False, aligned=True), Array(float64, 2, 'A', False, aligned=True))
  - beta[factor][0]


In [2]:
import pandas as pd
from finance.trading.factor.preprocess import *

max_lag = 250
n_factors = 10
path_price = "/home/georges/compet/orycterope/prices.csv"
prices = pd.read_csv(path_price, sep=";")
preprocessor = PreProcess(max_lag)
X_train0, Y_train0 = preprocessor.split_x_y(
    prices, DataType.PRICE
)
path_ret = "/home/georges/compet/data-challenge-qrt/2022/data.csv"
data = pd.read_csv(path_ret, sep=";")
X_train1, Y_train1 = preprocessor.split_x_y(
    data, DataType.RETURN
)

X_train, Y_train = X_train1.copy(), Y_train1.copy()

In [3]:
# from finance.trading.factor.benchmark import Benchmark

bench = Benchmark(
        lag=max_lag, n_factors=n_factors, n_iter=1500, eps=1e-6
    )
stiefel, beta = bench.train(X_train, Y_train)

Benchmark model:   0%|          | 0/1500 [00:00<?, ?it/s]

Iteration 0 with best train metric: 0.02331346933035975
Iteration 1 with best train metric: 0.03319696206337412
Iteration 3 with best train metric: 0.035302048607635694
Iteration 7 with best train metric: 0.03999544914662606
Iteration 67 with best train metric: 0.0435936893692612
Iteration 138 with best train metric: 0.044332621329282895
Iteration 496 with best train metric: 0.04441416345796451
Iteration 675 with best train metric: 0.04578782187329137
Iteration 1361 with best train metric: 0.05094188095529535


In [4]:
from finance.trading.factor.proj_grad import ProjGrad
proj = ProjGrad(
    lag=max_lag,
    n_factors=n_factors,
    eps=1.0e-6,
    step_stiefel=0.1,
    step_beta=0.05,
    random_state=1234,
    n_iter=10000,
    use_benchmark=False,
)

In [5]:
print(proj.metric_train(stiefel, beta, X_train, Y_train))

0.05094188095529535


In [6]:
stiefel_bis, beta_bis = proj.train(X_train, Y_train, stiefel, beta)

ProjGrad model:   0%|          | 0/10000 [00:00<?, ?it/s]

Iteration: 494 with best train metric: 0.0509684463662212
Iteration: 498 with best train metric: 0.05106057653208777
Iteration: 192 with best train metric: 0.05114563466064009
Iteration: 466 with best train metric: 0.051208702740273664
Iteration: 132 with best train metric: 0.05126493683224949
Iteration: 70 with best train metric: 0.05129162474275833
Iteration: 161 with best train metric: 0.05130867335066276
Iteration: 271 with best train metric: 0.05136999683579173
Iteration: 60 with best train metric: 0.051384578133798084
Iteration: 402 with best train metric: 0.05139800417477612
Iteration: 122 with best train metric: 0.05145472293014483
Iteration: 341 with best train metric: 0.051503196536216225
Iteration: 482 with best train metric: 0.05151066957174731
Iteration: 333 with best train metric: 0.05151984725561085
Iteration: 371 with best train metric: 0.05155813239387107
Iteration: 373 with best train metric: 0.051576716171193714
Iteration: 327 with best train metric: 0.05160127332422

In [7]:
print(proj.metric_train(stiefel_bis, beta_bis, X_train, Y_train))

0.13071463478068074


# Test 